In [1]:
# import the function file
from Workflow_Python_Functions import *

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [2]:
#Define input directory, keep all files in same directory and scripts so getwd works
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/MTBLS709_Data_32"
input_dir

'/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/MTBLS709_Data_32'

In [3]:
sp = sirius_postproc(input_dir, exp_int = 0.90, csi_score = -150)

IndexError: list index out of range

In [4]:
# entry is all files and folders in input_dir
entries = []
for entry in os.listdir(input_dir):
    if os.path.isdir(os.path.join(input_dir, entry)):
        print(entry)
        entries.append(entry)

pos_09_summer_Plaund_9_1-B,1_01_17180
pos_25_autumn_Marpol_25_2-A,6_01_17235
pos_26_summer_Marpol_26_1-D,2_01_17201
pos_03_spring_Fistax_3_1-A,3_01_17277
pos_08_spring_Polstr_8_1-A,8_01_17284
pos_05_summer_Polstr_5_1-A,5_01_17174
pos_02_spring_Fistax_2_1-A,2_01_17276
pos_27_summer_Marpol_27_1-D,3_01_17202
pos_12_spring_Plaund_12_1-B,4_01_17288
pos_08_autumn_Polstr_8_1-E,3_01_17214
pos_27_autumn_Marpol_27_2-A,8_01_17237
pos_02_summer_Fistax_2_1-A,2_01_17171
pos_27_spring_Marpol_27_1-D,3_01_17306
pos_25_summer_Marpol_25_1-D,1_01_17200
pos_13_autumn_Plaund_13_1-E,8_01_17219
pos_01_summer_Fistax_1_1-A,1_01_17170
pos_06_spring_Polstr_6_1-A,6_01_17280
pos_02_winter_Fistax_2_2-B,2_01_17241


In [5]:
entry = entries[15]
entry

'pos_01_summer_Fistax_1_1-A,1_01_17170'

In [6]:
sub_dir = input_dir + "/" + entry + '/insilico/SIRIUS/'
msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

In [9]:
if os.path.exists(msp_csv) and os.path.exists(sub_dir):
    #output json files from SIRIUS
    files_S = (glob.glob(sub_dir+'/*.json'))
    #list of precursor m/z
    msp = pd.read_csv(msp_csv)

    # for each mz
    for mz, row in msp.iterrows():
        # make a list of files with this mz
        files_for_mz = []

        for file in files_S:
            if str(msp["premz"][mz]) in file:
                files_for_mz.append(file)

        # in case if SL and ALL are given
        if len(files_for_mz)==2:

            # if the SL file is before the ALL file
            if len(files_for_mz[0])>len(files_for_mz[1]):

                # extract the formula and structure files
                json_dirSL = next(os.walk(files_for_mz[0]))[1]
                sub_sub_dirSL_structure_can = files_for_mz[0] + "/" + json_dirSL[0]  + "/structure_candidates.tsv"                   
                sub_sub_dirSL_formula_can = files_for_mz[0] + "/" + json_dirSL[0]  + "/formula_candidates.tsv" 
                SL_Canopus_csv = files_for_mz[0] + "/canopus_summary.tsv"


                # extract the formula and structure files
                json_dirALL = next(os.walk(files_for_mz[1]))[1]
                sub_sub_dirALL_structure_can = files_for_mz[1] + "/" + json_dirALL[0] +"/structure_candidates.tsv"
                sub_sub_dirALL_formula_can = files_for_mz[1] + "/" + json_dirALL[0] +"/formula_candidates.tsv"
                ALL_Canopus_csv = files_for_mz[1] + "/canopus_summary.tsv"


            # if the ALL file is before the SL file
            elif len(files_for_mz[1]) >len(files_for_mz[0]):


                # extract the formula and structure files
                json_dirALL = next(os.walk(files_for_mz[0]))[1]
                sub_sub_dirALL_structure_can = files_for_mz[0] + "/" + json_dirALL[0]  + "/structure_candidates.tsv"                   
                sub_sub_dirALL_formula_can = files_for_mz[0] + "/" + json_dirALL[0]  + "/formula_candidates.tsv"
                ALL_Canopus_csv = files_for_mz[0] + "/canopus_summary.tsv"


                # extract the formula and structure files
                json_dirSL = next(os.walk(files_for_mz[1]))[1]
                sub_sub_dirSL_structure_can = files_for_mz[1] + "/" + json_dirSL[0] +"/structure_candidates.tsv"
                sub_sub_dirSL_formula_can = files_for_mz[1] + "/" + json_dirALL[0]  + "/formula_candidates.tsv"
                SL_Canopus_csv = files_for_mz[1] + "/canopus_summary.tsv"

            # if both structure files exist and they have more than 0 rows
            if os.path.exists(sub_sub_dirSL_structure_can) and len(pd.read_csv(sub_sub_dirSL_structure_can, sep = "\t"))>0 and os.path.exists(sub_sub_dirALL_structure_can) and len(pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")):

                # read strcuture and formula tsv files for both SL and ALL
                SL_structure_csv = pd.read_csv(sub_sub_dirSL_structure_can, sep = "\t")
                SL_formula_csv = pd.read_csv(sub_sub_dirSL_formula_can, sep = "\t")
                SL_Canopus = pd.read_csv(SL_Canopus_csv, sep = "\t")


                ALL_structure_csv = pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")
                ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
                ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")

                # Add the structure and formula files together
                for structure, rows in ALL_structure_csv.iterrows():
                    for formula, rows in ALL_formula_csv.iterrows():
                        if ALL_structure_csv["formulaRank"][structure] == ALL_formula_csv["rank"][formula]:
                            ALL_structure_csv.loc[structure, 'SiriusScore'] = ALL_formula_csv['SiriusScore'][formula]
                            ALL_structure_csv.loc[structure, 'numExplainedPeaks'] = ALL_formula_csv['numExplainedPeaks'][formula]
                            ALL_structure_csv.loc[structure, 'explainedIntensity'] = ALL_formula_csv['explainedIntensity'][formula]
                            ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"
                            if ALL_formula_csv["molecularFormula"][formula] == ALL_Canopus["molecularFormula"][0]:
                                ALL_structure_csv.loc[structure, 'superclass'] = ALL_Canopus['superclass'][0]
                                ALL_structure_csv.loc[structure, 'class'] = ALL_Canopus['class'][0]
                                ALL_structure_csv.loc[structure, 'subclass'] = ALL_Canopus['subclass'][0]

                # Add the structure and formula files together
                for structure_sl, rows in SL_structure_csv.iterrows():
                    for formula_sl, rows in SL_formula_csv.iterrows():
                        if SL_structure_csv["formulaRank"][structure_sl] == SL_formula_csv["rank"][formula_sl]:
                            SL_structure_csv.loc[structure_sl, 'SiriusScore'] = SL_formula_csv['SiriusScore'][formula_sl]
                            SL_structure_csv.loc[structure_sl, 'numExplainedPeaks'] = SL_formula_csv['numExplainedPeaks'][formula_sl]
                            SL_structure_csv.loc[structure_sl, 'explainedIntensity'] = SL_formula_csv['explainedIntensity'][formula_sl]
                            SL_structure_csv.loc[structure_sl, "SuspectListEntry"] = "TRUE"   
                            if SL_formula_csv["molecularFormula"][formula_sl] == SL_Canopus["molecularFormula"][0]:
                                SL_structure_csv.loc[structure_sl, 'superclass'] = SL_Canopus['superclass'][0]
                                SL_structure_csv.loc[structure_sl, 'class'] = SL_Canopus['class'][0]
                                SL_structure_csv.loc[structure_sl, 'subclass'] = SL_Canopus['subclass'][0] 


                # after formula and structure have been merged, merge SL and ALL results
                all_sl_db = pd.concat([ALL_structure_csv, SL_structure_csv], ignore_index=True)
                for str_sirius, row in all_sl_db.iterrows():
                    if not str_can_score(all_sl_db, str_sirius):
                        all_sl_db = all_sl_db.drop(str_sirius, inplace=False)

                if not os.path.exists(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]):
                    os.mkdir(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1])

                result_sirius_name = (sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")
                msp.loc[mz, "sirius_result_dir"] = result_sirius_name.replace(input_dir, ".")
                all_sl_db.to_csv(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")

            # if only ALL structure file exists and they have more than 0 rows
            elif not (os.path.exists(sub_sub_dirSL_structure_can) and len(pd.read_csv(sub_sub_dirSL_structure_can, sep = "\t"))>0) and os.path.exists(sub_sub_dirALL_structure_can) and len(pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")):
                ALL_structure_csv = pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")
                ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
                ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")

                # Add the structure and formula files together
                for structure, rows in ALL_structure_csv.iterrows():
                    for formula, rows in ALL_formula_csv.iterrows():
                        if ALL_structure_csv["formulaRank"][structure] == ALL_formula_csv["rank"][formula]:
                            ALL_structure_csv.loc[structure, 'SiriusScore'] = ALL_formula_csv['SiriusScore'][formula]
                            ALL_structure_csv.loc[structure, 'numExplainedPeaks'] = ALL_formula_csv['numExplainedPeaks'][formula]
                            ALL_structure_csv.loc[structure, 'explainedIntensity'] = ALL_formula_csv['explainedIntensity'][formula]
                            ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"
                            if ALL_formula_csv["molecularFormula"][formula] == ALL_Canopus["molecularFormula"][0]:
                                ALL_structure_csv.loc[structure, 'superclass'] = ALL_Canopus['superclass'][0]
                                ALL_structure_csv.loc[structure, 'class'] = ALL_Canopus['class'][0]
                                ALL_structure_csv.loc[structure, 'subclass'] = ALL_Canopus['subclass'][0]

                for str_siriusA, row in ALL_structure_csv.iterrows():
                    if not str_can_score(ALL_structure_csv, str_siriusA):
                        ALL_structure_csv = ALL_structure_csv.drop(str_siriusA, inplace=False)


                if not os.path.exists(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]):
                    os.mkdir(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1])

                result_sirius_name = (sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")
                msp.loc[mz, "sirius_result_dir"] = result_sirius_name.replace(input_dir, ".")
                ALL_structure_csv.to_csv(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")

            elif not(os.path.exists(sub_sub_dirSL_structure_can) and len(pd.read_csv(sub_sub_dirSL_structure_can, sep = "\t"))>0 and os.path.exists(sub_sub_dirALL_structure_can) and len(pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t"))):
                if os.path.exists(sub_sub_dirALL_formula_can) and pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t"):
                    ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
                    ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")
                    for formula, rows in ALL_formula_csv.iterrows():
                        ALL_formula_csv.loc[formula, 'superclass'] = ALL_Canopus['superclass'][0]
                        ALL_formula_csv.loc[formula, 'class'] = ALL_Canopus['class'][0]
                        ALL_formula_csv.loc[formula, 'subclass'] = ALL_Canopus['subclass'][0]

                    for for_siriusA, row in ALL_formula_csv.iterrows():
                        if not ALL_formula_csv['explainedIntensity'][for_siriusA] >= exp_int:
                            ALL_formula_csv = ALL_formula_csv.drop(for_siriusA, inplace=False)
                    if not os.path.exists(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]):
                        os.mkdir(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1])

                    result_sirius_name = (sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"formula_"+json_dirALL[0].split("_")[-1] + ".csv")
                    msp.loc[mz, "sirius_result_dir"] = result_sirius_name.replace(input_dir, ".")

                    ALL_formula_csv.to_csv(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"formula_"+json_dirALL[0].split("_")[-1] + ".csv")

                else:
                    pass
            else:
                pass

        elif len(files_for_mz) == 1:
            # extract the formula and structure files
            json_dirALL = next(os.walk(files_for_mz[0]))[1]
            
            if len(json_dirALL)>0:
            
                sub_sub_dirALL_structure_can = files_for_mz[0] + "/" + json_dirALL[0]  + "/structure_candidates.tsv"                   
                sub_sub_dirALL_formula_can = files_for_mz[0] + "/" + json_dirALL[0]  + "/formula_candidates.tsv" 
                ALL_Canopus_csv = files_for_mz[1] + "/canopus_summary.tsv"

                # if both structure files exist
                if os.path.exists(sub_sub_dirALL_structure_can) and len(pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t"))>0:
                    ALL_structure_csv = pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")
                    ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
                    ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")
                    # Add the structure and formula files together
                    for structure, rows in ALL_structure_csv.iterrows():
                        for formula, rows in ALL_formula_csv.iterrows():
                            if ALL_structure_csv["formulaRank"][structure] == ALL_formula_csv["rank"][formula]:
                                ALL_structure_csv.loc[structure, 'SiriusScore'] = ALL_formula_csv['SiriusScore'][formula]
                                ALL_structure_csv.loc[structure, 'numExplainedPeaks'] = ALL_formula_csv['numExplainedPeaks'][formula]
                                ALL_structure_csv.loc[structure, 'explainedIntensity'] = ALL_formula_csv['explainedIntensity'][formula]
                                ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"

                                if ALL_formula_csv["molecularFormula"][formula] == ALL_Canopus["molecularFormula"][0]:
                                    ALL_structure_csv.loc[structure, 'superclass'] = ALL_Canopus['superclass'][0]
                                    ALL_structure_csv.loc[structure, 'class'] = ALL_Canopus['class'][0]
                                    ALL_structure_csv.loc[structure, 'subclass'] = ALL_Canopus['subclass'][0]

                    for str_siriusA, row in ALL_structure_csv.iterrows():
                        if not str_can_score(ALL_structure_csv, str_siriusA):
                            ALL_structure_csv = ALL_structure_csv.drop(str_siriusA, inplace=False)
                    if not os.path.exists(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]):
                        os.mkdir(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1])

                    result_sirius_name = (sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")
                    msp.loc[mz, "sirius_result_dir"] = result_sirius_name.replace(input_dir, ".")

                    ALL_structure_csv.to_csv(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"structure_"+json_dirALL[0].split("_")[-1] + ".csv")


                elif not (os.path.exists(sub_sub_dirALL_structure_can) and len(pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t"))):
                    if os.path.exists(sub_sub_dirALL_formula_can) and pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t"):
                        ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
                        ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")
                        for formula, rows in ALL_formula_csv.iterrows():
                            ALL_formula_csv.loc[formula, 'superclass'] = ALL_Canopus['superclass'][0]
                            ALL_formula_csv.loc[formula, 'class'] = ALL_Canopus['class'][0]
                            ALL_formula_csv.loc[formula, 'subclass'] = ALL_Canopus['subclass'][0]

                        for for_siriusA, row in ALL_formula_csv.iterrows():
                            if not ALL_formula_csv['explainedIntensity'][for_siriusA] >= exp_int:
                                ALL_formula_csv = ALL_formula_csv.drop(for_siriusA, inplace=False)
                        if not os.path.exists(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]):
                            os.mkdir(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1])

                        result_sirius_name = (sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"formula_"+json_dirALL[0].split("_")[-1] + ".csv")
                        msp.loc[mz, "sirius_result_dir"] = result_sirius_name.replace(input_dir, ".")

                        ALL_formula_csv.to_csv(sub_dir+"results_for_"+json_dirALL[0].split("_")[-1]+"_"+"formula_"+json_dirALL[0].split("_")[-1] + ".csv")

                    else:
                        pass
                else:
                    pass
        msp.to_csv(msp_csv)


In [8]:
json_dirALL

[]